In [1]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from fastai.vision import *
from fastai import *
from fastai.basics import *
from fastai.callback.all import *
from fastai.vision.all import *
from fastai.medical.imaging import *

import pydicom

import pandas as pd

from pathlib import Path
from shutil import copy

import warnings
warnings.filterwarnings("ignore")


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.7/site-packages/kornia/augmentation/augmentation.py:1875: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  category=DeprecationWarning,


In [ ]:
# csv = pd.read_csv('../input/vinbigdata-1024-image-dataset/vinbigdata/train.csv')
# csv.head()

# twoclasscsv = pd.DataFrame(columns=['file', 'label'])

# csvdict = {}

# for index, row in csv.iterrows():
#     csvdict[row['image_id']] = (int(row['class_id']) == 14)
    
# for key in csvdict:
#     print(key, csvdict[key])
#     break

# for key in csvdict:
#     twoclasscsv.loc[len(twoclasscsv)] = [key+'.png', ('Normal' if csvdict[key] else 'Abnormal')]
    
# twoclasscsv.head()

# twoclasscsv.to_csv("./two-class-train.csv")

In [2]:
df = pd.read_csv('../input/vinbigdata-2-class-csv/two-class-train.csv')
df.drop('Unnamed: 0', inplace=True, axis=1)
df.head()

,file,label
0,50a418190bc3fb1ef1633bf9678929b3.png,Normal
1,21a10246a5ec7af151081d0cd6d65dc9.png,Normal
2,9a5094b2563a1ef3ff50dc5c7ff71345.png,Abnormal
3,051132a778e61a86eb147c7c6f564dfe.png,Abnormal
4,063319de25ce7edb9b1c6b8881290140.png,Normal


In [3]:
!mkdir ./Normal/
!mkdir ./Abnormal/

In [4]:
for index, row in df.iterrows():
    copy('../input/vinbigdata-1024-image-dataset/vinbigdata/train/'+row['file'], './'+row['label']+'/')

In [5]:
tfms = aug_transforms(do_flip = True, flip_vert = False, mult=2.0)


In [6]:
np.random.seed(42)
data = ImageDataLoaders.from_folder('./',train = "", valid_pct=0.2, item_tfms=Resize(1024), batch_tfms=tfms, bs = 4, num_workers = 4)

In [7]:
Data = DataBlock( blocks=(ImageBlock, CategoryBlock), get_items=get_image_files, 
splitter=RandomSplitter(valid_pct=0.2, seed=42), get_y=parent_label, item_tfms=Resize(1024))
dls = Data.dataloaders('./')

In [8]:
learn = cnn_learner(data, resnet34, metrics=error_rate)
learn.fine_tune(0)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,0.656889,1.539252,0.285667,11:33


In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1, figsize = (25,5))

In [11]:
learn.save('export1')

Path('models/export1.pth')

In [12]:
from google.colab import files

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
learn.unfreeze()

In [ ]:
def find_appropriate_lr(model:Learner, lr_diff:int = 15, loss_threshold:float = .05, adjust_value:float = 1, plot:bool = False) -> float:
    #Run the Learning Rate Finder
    model.lr_find()
    
    #Get loss values and their corresponding gradients, and get lr values
    losses = np.array(model.recorder.losses)
    assert(lr_diff < len(losses))
    loss_grad = np.gradient(losses)
    lrs = model.recorder.lrs
    
    #Search for index in gradients where loss is lowest before the loss spike
    #Initialize right and left idx using the lr_diff as a spacing unit
    #Set the local min lr as -1 to signify if threshold is too low
    r_idx = -1
    l_idx = r_idx - lr_diff
    while (l_idx >= -len(losses)) and (abs(loss_grad[r_idx] - loss_grad[l_idx]) > loss_threshold):
        local_min_lr = lrs[l_idx]
        r_idx -= 1
        l_idx -= 1

    lr_to_use = local_min_lr * adjust_value
    
    if plot:
        # plots the gradients of the losses in respect to the learning rate change
        plt.plot(loss_grad)
        plt.plot(len(losses)+l_idx, loss_grad[l_idx],markersize=10,marker='o',color='red')
        plt.ylabel("Loss")
        plt.xlabel("Index of LRs")
        plt.show()

        plt.plot(np.log10(lrs), losses)
        plt.ylabel("Loss")
        plt.xlabel("Log 10 Transform of Learning Rate")
        loss_coord = np.interp(np.log10(lr_to_use), np.log10(lrs), losses)
        plt.plot(np.log10(lr_to_use), loss_coord, markersize=10,marker='o',color='red')
        plt.show()
        
    return lr_to_use

In [ ]:
lr = find_appropriate_lr(learn, plot = True)

In [ ]:
learn.fit_one_cycle(5, lr=lr)

In [ ]:
learn.export()

In [ ]:
test_csv = pd.read_csv('../input/vinbigdata-1024-image-dataset/vinbigdata/train.csv')

In [ ]:
for index, row in test_csv.iterrows():
    cat, tensor, probs = learn.predict(open_image(f"../input/vinbigdata-1024-image-dataset/vinbigdata/test/{row['image_id']}"))